##  Load data into Environment

In [ ]:
import numpy as np 
import pandas as pd


sms_data=pd.read_csv('spam.csv',encoding='latin-1')

print(sms_data.head())

cols=sms_data.columns[:2]
data=sms_data[cols]
print(data.shape)
data=data.rename(columns={"v1":"Value",'v2':'Text'})

print(data.Value.value_counts())

# Feature Engineering

In [ ]:
import re
import nltk
from nltk import word_tokenize
from string import punctuation

punctuation=list(punctuation)

# Creating a new feature called Punctuations. 
# This feature counts the number of punctuation characters in the sms message 
data['Punctuations']=data['Text'].apply(lambda x:len(re.findall(r'[^\w+&&\s]',x)))

# Creating a new feature called Phonenumbers. 
# This feature indicates if the sms text contains a phonenumber or not
data['PhoneNumbers']=data['Text'].apply(lambda x:re.findall(r'[\d]{10}',x))

# Creating a new feature called Links.
# This feature indicates if the sms text contains a URL or not 

is_link= lambda x: 1 if re.search(r'https?://([-\w.]|(?:%[\da-fA-F]{2}))+',x)!=0 else 0
data['Links']=data['Text'].apply(is_link)


# Creating a new feature called Uppercase.
# This feature indicates how many words in the the sms text are in upper case

count_upper=lambda x: list(map(str.isupper,x.split())).count(True)

data['Uppercase']=data['Text'].apply(count_upper)


def find_unusual_words(text):
    text_vocab_set=set(w.lower() for w in text if w.isalpha())
    english_vocab_set=set(w.lower() for w in nltk.corpus.words.words())
    
    unusual_set=text_vocab_set-english_vocab_set
    return len(unusual_set)
data['unusualwords']=data['Text'].apply(lambda x:find_unusual_words(word_tokenize(x)))
    
    

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf=TfidfVectorizer(stop_words='english',strip_accents='ascii',max_features=500)

tf_idf_matrix=tf_idf.fit_transform(data['Text'])

data_extra_features=pd.concat([data,pd.DataFrame(tf_idf_matrix.toarray(),columns=tf_idf_matrix.get_feature_names))],axis=1)


## machine Learning

In [ ]:
from sklearn.model_selection import train_test_split
X=data_extra_features
features=X.columns.drop["Value","Text","Value_num"]
target=['Value']
X_train,X_test,y_train,y_test=train_test_split(X[features],X[target])

#Decision Tree
from sklearn.tree import DecisionTreeClassifer
from sklearn.metrics import accuracy_score


def model_accuray(model,X_train,X_test,y_train,y_test):
    
    model.fit(X_train,y_train)
    pred=dt.predict(X_test)
    print('Traning accurarcy',accuracy_score(y_train,dt.preditct(X_train)))
    print('Testing accurarcy',accuracy_score(y_test,pred))

from sklearn.navie_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
#NavieBayes model

mb=MultinomialNB()
lr=LogisticRegression()
dt=DecisionTreeClassifier(min_samples_split=40)

model_accuray(mb,X_train,X_test,y_train,y_test)
model_accuray(lr,X_train,X_test,y_train,y_test)
model_accuray(dt,X_train,X_test,y_train,y_test)
